In [ ]:
import os
import numpy as np
import cv2
from pycocotools.coco import COCO
from pycocotools import mask as maskUtils
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
# # Paths
# annotation_file = 'wildfire-week3-v3/test/_annotations.coco.json'  # Replace with your annotations file
# image_dir = 'wildfire-week3-v3/test/original'                       # Replace with your images directory
# mask_dir = 'wildfire-week3-v3/test/mask'            # Directory to save masks
def generate_mask(annotation_file, image_dir, mask_dir): 
    os.makedirs(mask_dir, exist_ok=True)
    # Initialize COCO API
    coco = COCO(annotation_file)

# Get all image IDs
    image_ids = coco.getImgIds()

# Define category mapping
    categories = coco.loadCats(coco.getCatIds())
    category_mapping = {cat['id']: cat['name'] for cat in categories}
    num_classes = len(category_mapping)
    print(f"Number of classes: {num_classes}")

    for image_id in tqdm(image_ids, desc="Generating Multi-Class Masks"):
    # Load image info
        image_info = coco.loadImgs(image_id)[0]
        height, width = image_info['height'], image_info['width']

    # Initialize blank multi-class mask
        multi_class_mask = np.zeros((height, width), dtype=np.uint8)

    # Get all annotations for the image
        ann_ids = coco.getAnnIds(imgIds=image_id)
        annotations = coco.loadAnns(ann_ids)

        for ann in annotations:
            category_id = ann['category_id']
            ann_id = ann['id']

        # Generate mask for the annotation
            if ann['iscrowd']:
                rle = ann['segmentation']
                mask = maskUtils.decode(rle)
            else:
                mask = coco.annToMask(ann)

        # Assign category_id to mask pixels
        # Handle overlapping by assigning the category_id (latest object overwrites)
            multi_class_mask[mask > 0] = category_id

    # Save the multi-class mask as PNG
        image_filename = image_info['file_name']
        mask_filename = os.path.splitext(image_filename)[0] + '_mask.png'
        mask_path = os.path.join(mask_dir, mask_filename)
        cv2.imwrite(mask_path, multi_class_mask)

    
# Create mask directory if it doesn't exist


    # Optional: Visualize the mask overlay
    '''
    # Load original image
    image_path = os.path.join(image_dir, image_filename)
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Create color map
    unique_categories = np.unique(multi_class_mask)
    unique_categories = unique_categories[unique_categories != 0]  # Exclude background
    num_unique = len(unique_categories)
    colors = plt.cm.get_cmap('hsv', num_unique + 1)

    color_map = {}
    for idx, category_id in enumerate(unique_categories, start=1):
        color_map[category_id] = (np.array(colors(idx)[:3]) * 255).astype(np.uint8)

    # Create color mask
    color_mask = np.zeros((height, width, 3), dtype=np.uint8)
    for category_id, color in color_map.items():
        color_mask[multi_class_mask == category_id] = color

    # Overlay mask on image
    overlay = cv2.addWeighted(image_rgb, 0.7, color_mask, 0.3, 0)

    # Display
    plt.figure(figsize=(12, 12))
    plt.imshow(overlay)
    plt.axis('off')
    plt.title(f"Image ID: {image_id} with Mask Overlay")
    plt.show()
    '''


In [ ]:
# Directories for your datasets
train_annotation_file = 'wildfire-week4/train/_annotations.coco.json'
train_images_dir = 'wildfire-week4/train/original'
train_masks_dir = 'wildfire-week4/train/mask'

valid_annotation_file = 'wildfire-week4/valid/_annotations.coco.json'
valid_images_dir = 'wildfire-week4/valid/original'
valid_masks_dir = 'wildfire-week4/valid/mask'

test_annotation_file = 'wildfire-week4/test/_annotations.coco.json'
test_images_dir = 'wildfire-week4/test/original'
test_masks_dir = 'wildfire-week4/test/mask'

In [ ]:
generate_mask(train_annotation_file, train_images_dir, train_masks_dir)

In [ ]:
generate_mask(valid_annotation_file, valid_images_dir, valid_masks_dir)

In [ ]:
generate_mask(test_annotation_file, test_images_dir, test_masks_dir)

In [ ]:
import torch
from segmentation_models_pytorch import Unet

# Initialize the model
unet_model = Unet(
    encoder_name="resnet50",  # Choose your backbone (e.g., "resnet34", "efficientnet-b3", etc.)
    encoder_weights="imagenet",  # Use pre-trained weights
    in_channels=3,  # Input channels (e.g., 3 for RGB images)
    classes=5  # Number of output classes
)

# Example input
input_tensor = torch.randn(1, 3, 640, 640)  # Batch size of 1, 3 channels (RGB), 640x640 image
output = unet_model(input_tensor)
print(output.shape)

In [ ]:
from torch.utils.data import Dataset
class SegmentationDataset(Dataset):
    def __init__(self, images_dir, masks_dir, transform=None):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.transform = transform
        self.image_filenames = os.listdir(images_dir)

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        image_path = os.path.join(self.images_dir, self.image_filenames[idx])
        mask_path = os.path.join(self.masks_dir, self.image_filenames[idx].replace('.jpg', '')+'_mask.png')  # Adjust file extension if needed

        image = cv2.imread(image_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Debugging prints
        if image is None:
            raise ValueError(f"Image not found or unable to load: {image_path}")
        if mask is None:
            raise ValueError(f"Mask not found or unable to load: {mask_path}")

        # Ensure proper dimensions
        if len(image.shape) != 3 or image.shape[2] != 3:
            raise ValueError(f"Unexpected image shape: {image.shape}")

        # Resize or pad image and mask to be divisible by 32
        # image = self.resize_or_pad(image)
        # mask = self.resize_or_pad(mask)

        # Normalize image
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # mask = mask // 51  # Adjust if necessary

        if self.transform:
            image = self.transform(image)

        return torch.tensor(image, dtype=torch.float32).permute(0, 1, 2), torch.tensor(mask, dtype=torch.long)


In [ ]:
from torchvision import transforms
from torch.utils.data import DataLoader

# Define transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((640, 640)),
    transforms.ToTensor(),
])

# # Directories for your datasets
# train_images_dir = 'wildfire-week3-v3/train/original'
# train_masks_dir = 'wildfire-week3-v3/train/mask'
# valid_images_dir = 'wildfire-week3-v3/valid/original'
# valid_masks_dir = 'wildfire-week3-v3/valid/mask'
# test_images_dir = 'wildfire-week3-v3/test/original'
# test_masks_dir = 'wildfire-week3-v3/test/mask'

# Create datasets
train_dataset = SegmentationDataset(train_images_dir, train_masks_dir, transform)
valid_dataset = SegmentationDataset(valid_images_dir, valid_masks_dir, transform)
test_dataset = SegmentationDataset(test_images_dir, test_masks_dir, transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=4, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)


In [ ]:
import torch.optim as optim
from tqdm import tqdm
import copy
import torch

# Initialize model, loss function, and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(unet_model.parameters(), lr=1e-4)

# Early stopping parameters
patience = 3  # Number of epochs to wait for improvement
best_model_path = "wildfire-week4/best_model.pth"  # Path to save the best model

# Training loop with early stopping, saving, and restoring the best model
def train_model(model, train_loader, valid_loader, criterion, optimizer, num_epochs=10, patience=3):
    best_valid_loss = float('inf')
    early_stopping_counter = 0
    best_model_weights = copy.deepcopy(model.state_dict())  # Copy initial model weights

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, masks in tqdm(train_loader):
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}")

        # Validation
        model.eval()
        valid_loss = 0.0
        with torch.no_grad():
            for images, masks in valid_loader:
                outputs = model(images)
                loss = criterion(outputs, masks)
                valid_loss += loss.item()

        valid_loss = valid_loss / len(valid_loader)
        print(f"Validation Loss: {valid_loss:.4f}")

        # Check for improvement in validation loss
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            early_stopping_counter = 0  # Reset counter if validation loss improves
            best_model_weights = copy.deepcopy(model.state_dict())  # Store best weights
            torch.save(best_model_weights, best_model_path)  # Save best model to disk
            print(f"Validation loss improved to {valid_loss:.4f}. Best model saved to {best_model_path}.")
        else:
            early_stopping_counter += 1
            print(f"No improvement in validation loss for {early_stopping_counter} epoch(s).")

        # Early stopping condition
        if early_stopping_counter >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break

    # Restore the best model weights after early stopping
    model.load_state_dict(torch.load(best_model_path))
    print("Best model weights restored from saved model.")






In [ ]:
# Train the model with early stopping, saving, and restoring best weights
train_model(unet_model, train_loader, valid_loader, torch.nn.CrossEntropyLoss(), optimizer, num_epochs=10, patience=3)

In [ ]:
import torch
import numpy as np
import pandas as pd

def evaluate_model_classname(model, data_loader, criterion, class_names):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0.0
    total_pixels = 0
    correct_predictions = 0

    # Initialize counters for each class
    num_classes = len(class_names)
    class_correct = torch.zeros(num_classes)  # Correct predictions per class
    class_total = torch.zeros(num_classes)    # Total ground truth pixels per class
    class_predicted = torch.zeros(num_classes)  # Total predicted pixels per class

    with torch.no_grad():  # Disable gradient calculation
        for images, labels in data_loader:
            preds = model(images)

            # Ensure labels are of shape [batch_size, height, width]
            if labels.dim() == 4:  # If labels have an extra dimension
                labels = labels.squeeze(1)  # Remove the channel dimension

            # Convert labels to Long type (int64)
            labels = labels.long()

            # Calculate loss
            loss = criterion(preds, labels)
            total_loss += loss.item()

            # Get the predicted class for each pixel
            _, predicted = torch.max(preds, 1)  # Shape: [batch_size, height, width]

            # Update overall accuracy (correct_predictions / total_pixels)
            correct_predictions += (predicted == labels).sum().item()
            total_pixels += labels.numel()  # Total number of pixels in the batch

            # Update class-wise correct and total counts
            for class_index in range(num_classes):
                # Correct pixels for each class
                class_correct[class_index] += ((predicted == class_index) & (labels == class_index)).sum().item()
                # Total ground truth pixels for each class
                class_total[class_index] += (labels == class_index).sum().item()
                # Total predicted pixels for each class
                class_predicted[class_index] += (predicted == class_index).sum().item()

    # Calculate average loss and overall pixel accuracy
    average_loss = total_loss / len(data_loader)
    pixel_accuracy = correct_predictions / total_pixels

    # Calculate pixel accuracy for each class, avoiding division by zero
    class_pixel_accuracy = class_correct / class_total
    class_pixel_accuracy[class_total == 0] = 0  # Set accuracy to 0 for classes with no pixels

    # Calculate precision and recall for each class
    precision = np.zeros(num_classes)
    recall = np.zeros(num_classes)

    for class_index in range(num_classes):
        if class_total[class_index] > 0:  # Avoid division by zero
            # Precision: True Positives / (True Positives + False Positives)
            precision[class_index] = class_correct[class_index] / class_predicted[class_index] if class_predicted[class_index] > 0 else 0
            # Recall: True Positives / (True Positives + False Negatives)
            recall[class_index] = class_correct[class_index] / class_total[class_index]

    # Create a DataFrame to display results, excluding the first class
    results_df = pd.DataFrame({
        'Class Name': class_names[1:],  # Exclude the first class (background)
        'Pixel Accuracy': class_pixel_accuracy[1:].numpy(),  # Exclude the first class
        'Precision': precision[1:],  # Exclude the first class
        'Recall': recall[1:],  # Exclude the first class
    })

    # Round to three decimal places
    results_df[['Pixel Accuracy', 'Precision', 'Recall']] = results_df[['Pixel Accuracy', 'Precision', 'Recall']].round(3)

    return average_loss, pixel_accuracy, results_df


In [ ]:
import torch
import numpy as np
import pandas as pd

def evaluate_model(model, data_loader, criterion, class_names):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0.0
    total_pixels = 0
    correct_predictions = 0

    # Initialize counters for each class
    num_classes = len(class_names)
    class_correct = torch.zeros(num_classes)  # Correct predictions per class
    class_total = torch.zeros(num_classes)    # Total ground truth pixels per class
    class_predicted = torch.zeros(num_classes)  # Total predicted pixels per class

    with torch.no_grad():  # Disable gradient calculation
        for images, labels in data_loader:
            preds = model(images)

            # Ensure labels are of shape [batch_size, height, width]
            if labels.dim() == 4:  # If labels have an extra dimension
                labels = labels.squeeze(1)  # Remove the channel dimension

            # Convert labels to Long type (int64)
            labels = labels.long()

            # Calculate loss
            loss = criterion(preds, labels)
            total_loss += loss.item()

            # Get the predicted class for each pixel
            _, predicted = torch.max(preds, 1)  # Shape: [batch_size, height, width]

            # Update overall accuracy (correct_predictions / total_pixels)
            correct_predictions += (predicted == labels).sum().item()
            total_pixels += labels.numel()  # Total number of pixels in the batch

            # Update class-wise correct and total counts
            for class_index in range(num_classes):
                # Correct pixels for each class
                class_correct[class_index] += ((predicted == class_index) & (labels == class_index)).sum().item()
                # Total ground truth pixels for each class
                class_total[class_index] += (labels == class_index).sum().item()
                # Total predicted pixels for each class
                class_predicted[class_index] += (predicted == class_index).sum().item()

    # Calculate average loss and overall pixel accuracy
    average_loss = total_loss / len(data_loader)
    pixel_accuracy = correct_predictions / total_pixels

    # Calculate pixel accuracy for each class, avoiding division by zero
    class_pixel_accuracy = class_correct / class_total
    class_pixel_accuracy[class_total == 0] = 0  # Set accuracy to 0 for classes with no pixels

    # Calculate precision and recall for each class
    precision = np.zeros(num_classes)
    recall = np.zeros(num_classes)

    for class_index in range(num_classes):
        if class_total[class_index] > 0:  # Avoid division by zero
            # Precision: True Positives / (True Positives + False Positives)
            precision[class_index] = class_correct[class_index] / class_predicted[class_index] if class_predicted[class_index] > 0 else 0
            # Recall: True Positives / (True Positives + False Negatives)
            recall[class_index] = class_correct[class_index] / class_total[class_index]

    # Create a DataFrame to display results, excluding the first class
    results_df = pd.DataFrame({
        'Class Name': class_names[1:],  # Exclude the first class (background)
        'Pixel Accuracy': class_pixel_accuracy[1:].numpy(),  # Exclude the first class
        'Precision': precision[1:],  # Exclude the first class
        'Recall': recall[1:],  # Exclude the first class
    })

    # Round to three decimal places
    results_df[['Pixel Accuracy', 'Precision', 'Recall']] = results_df[['Pixel Accuracy', 'Precision', 'Recall']].round(3)

    return average_loss, pixel_accuracy, results_df


In [ ]:
num_classes = 5  # Adjust this according to your number of classes
average_loss, pixel_accuracy, class_pixel_accuracy, precision, recall, f1_score = evaluate_model(unet_model, test_loader, torch.nn.CrossEntropyLoss(), num_classes)

print(f'Validation Loss: {average_loss:.4f}')
print(f'Pixel Accuracy (Overall): {pixel_accuracy * 100:.4f}%')
for i in range(num_classes):
    print(f'Pixel Accuracy for Class {i}: {class_pixel_accuracy[i] * 100:.4f}%')
    print(f'Precision for Class {i}: {precision[i] * 100:.4f}%')
    print(f'Recall for Class {i}: {recall[i] * 100:.4f}%')
    print(f'F1 Score for Class {i}: {f1_score[i] * 100:.4f}%')

It has better accuracy for class 1, 2 and 3 and when epoch = 1 than ephch  = 10. So we will apply a weigth to fix imbalance class category problems

In [ ]:
import json
from collections import defaultdict
import os
from PIL import Image
import numpy as np

def count_classes_in_coco(coco_json_file, image_dir):
    """
    Counts the occurrences of each class (category) in the COCO dataset annotations, including the background.
    
    Args:
        coco_json_file (str): Path to the COCO format JSON annotation file.
        image_dir (str): Path to the directory containing the original images (to calculate background pixels).
        
    Returns:
        dict: A dictionary with class names as keys and their respective counts (in pixels) as values.
    """
    # Load the COCO JSON annotation file
    with open(coco_json_file, 'r') as f:
        coco_data = json.load(f)
    
    # Create a dictionary to store class counts
    class_counts = defaultdict(int)
    
    # Category mapping (id -> category name)
    category_mapping = {category['id']: category['name'] for category in coco_data['categories']}
    
    # To track pixels covered by objects
    covered_pixels_by_image = defaultdict(int)

    # Loop through all annotations
    for annotation in coco_data['annotations']:
        # Get the class id from the annotation (category_id)
        class_id = annotation['category_id']
        
        # Get the image id
        image_id = annotation['image_id']
        
        # Get the area of the segmented mask (to count the number of pixels)
        class_counts[class_id] += annotation['area']  # 'area' represents the number of pixels for the object
        
        # Accumulate the number of pixels covered by objects in each image
        covered_pixels_by_image[image_id] += annotation['area']
    
    # Include background calculation by analyzing total pixels in the images
    for image_info in coco_data['images']:
        image_id = image_info['id']
        image_path = os.path.join(image_dir, image_info['file_name'])
        
        # Open the image and calculate the total number of pixels
        image = Image.open(image_path)
        total_pixels = image.width * image.height
        
        # Calculate background pixels
        background_pixels = total_pixels - covered_pixels_by_image[image_id]
        
        # Add background class count
        class_counts[0] += background_pixels  # Assuming class 0 represents background

        
        keys = list(class_counts.keys())
        keys.sort()
        sorted_dict = {i: class_counts[i] for i in keys}
    return sorted_dict

# Example usage:
coco_json_path = "wildfire-week4/train/_annotations.coco.json"
image_directory = "wildfire-week4/train/original"
class_counts = count_classes_in_coco(coco_json_path, image_directory)
print(class_counts)


In [ ]:
import torch
import numpy as np

def calculate_class_weights(class_counts):
    """
    Calculate class weights based on the class counts from the dataset.
    
    Args:
        class_counts (dict): A dictionary with class IDs or names as keys and pixel counts as values.
    
    Returns:
        torch.Tensor: A tensor of class weights to be used in nn.CrossEntropyLoss.
    """
    # Get the total number of pixels across all classes
    total_pixels = sum(class_counts.values())

    # Calculate the frequency of each class (number of pixels of class / total pixels)
    class_frequencies = {cls: count / total_pixels for cls, count in class_counts.items()}

    # Invert the frequencies to give higher weights to less frequent classes
    class_weights = {cls: 1.0 / freq if freq > 0 else 0.0 for cls, freq in class_frequencies.items()}

    # Normalize weights so that they sum to 1 or use them directly
    weight_values = list(class_weights.values())
    weights_tensor = torch.tensor(weight_values, dtype=torch.float32)

    return weights_tensor



# Calculate weights based on class counts
class_weights = calculate_class_weights(class_counts)
print("Class Weights:", class_weights)

# You can now use these weights in nn.CrossEntropyLoss
criterion = torch.nn.CrossEntropyLoss(weight=class_weights)


In [ ]:
train_model(unet_model, train_loader, valid_loader, criterion, optimizer, num_epochs=10, patience=5)

In [ ]:
num_classes = 5  # Adjust this according to your number of classes
average_loss, pixel_accuracy, class_pixel_accuracy, precision, recall, f1_score = evaluate_model(unet_model, test_loader, torch.nn.CrossEntropyLoss(), num_classes)

print(f'Validation Loss: {average_loss:.4f}')
print(f'Pixel Accuracy (Overall): {pixel_accuracy * 100:.4f}%')
for i in range(num_classes):
    print(f'Pixel Accuracy for Class {i}: {class_pixel_accuracy[i] * 100:.4f}%')
    print(f'Precision for Class {i}: {precision[i] * 100:.4f}%')
    print(f'Recall for Class {i}: {recall[i] * 100:.4f}%')
    print(f'F1 Score for Class {i}: {f1_score[i] * 100:.4f}%')

In [ ]:
class_names = ['Background', 'Beetle Trees', 'Dead Tree', 'Debris', 'Water']  # Adjust class names as needed
average_loss, pixel_accuracy, results_table = evaluate_model_classname(unet_model, test_loader,torch.nn.CrossEntropyLoss() , class_names)
print(results_table)


In [ ]:
import torch
import random
import matplotlib.pyplot as plt


# Function to apply color mask over an image
def apply_mask(image, mask, alpha=0.3):
    """
    Apply a color mask over the image.
    
    Args:
        image (numpy.ndarray): Original image (H, W, 3).
        mask (numpy.ndarray): Mask to overlay (H, W).
        alpha (float): Transparency level for overlay.
        
    Returns:
        numpy.ndarray: Image with mask overlay.
    """
    # Create a color map for visualization, assuming 4 classes + background
    color_map = np.array([
        [0, 0, 0],        # background
        [255, 0, 0],      # class 1 (beetles tree) - Red
        [0, 255, 0],      # class 2 (dead tree) - Green
        [0, 0, 255],      # class 3 (debris) - Blue
        [255, 255, 0]     # class 4 (water) - Yellow
    ])

    colored_mask = color_map[mask]
    overlay = (image * (1 - alpha) + colored_mask * alpha).astype(np.uint8)
    return overlay

def visualize_predictions_with_ytrue(test_loader, model, num_samples=3):
    """
    Visualizes random predictions from the test loader with ground truth masks and 
    overlays both ground truth and predicted masks on the original image.
    
    Args:
        test_loader (DataLoader): DataLoader for test dataset.
        model (torch.nn.Module): Trained model for prediction.
        num_samples (int): Number of random samples to visualize.
        
    Returns:
        List of tuples: Each tuple contains (original_image, y_true_overlay, pred_overlay)
    """
    model.eval()  # Set model to evaluation mode
    output_list = []

    # Select a random batch from test_loader
    images, true_masks = next(iter(test_loader))
    
    # Get model predictions
    with torch.no_grad():
        outputs = model(images)
    
    # Convert predictions to class labels
    _, preds = torch.max(outputs, dim=1)
    
    # Move tensors back to numpy for visualization
    images = images.numpy().transpose(0, 2, 3, 1)  # Convert to (N, H, W, C)
    preds = preds.numpy()                          # Predicted masks
    true_masks = true_masks.numpy()                # Ground truth masks

    # Randomly select indices to visualize
    selected_indices = random.sample(range(images.shape[0]), num_samples)
    
    # Iterate over selected indices
    for idx in selected_indices:
        image = (images[idx] * 255).astype(np.uint8)  # Rescale image back to 0-255 range
        true_mask = true_masks[idx]
        pred_mask = preds[idx]

        # Apply ground truth mask and predicted mask over original image
        y_true_overlay = apply_mask(image, true_mask)
        pred_overlay = apply_mask(image, pred_mask)

        # Store the original image and overlays in a tuple
        output_list.append((image, y_true_overlay, pred_overlay))

        # Plot for visualization
        fig, axs = plt.subplots(1, 3, figsize=(15, 5))

        axs[0].imshow(image)
        axs[0].set_title("Original Image")
        axs[0].axis('off')

        axs[1].imshow(y_true_overlay)
        axs[1].set_title("Y_true Overlay")
        axs[1].axis('off')

        axs[2].imshow(pred_overlay)
        axs[2].set_title("Predicted Overlay")
        axs[2].axis('off')

        plt.show()

    return output_list

# Example usage:
output_list = visualize_predictions_with_ytrue(test_loader, unet_model, num_samples=3)


In [ ]:
import cv2
import numpy as np

def watershed_segmentation(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Remove noise
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)

    # Sure background area
    sure_bg = cv2.dilate(opening, kernel, iterations=3)

    # Sure foreground area
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    ret, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)

    # Unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)

    # Marker labelling
    ret, markers = cv2.connectedComponents(sure_fg)

    # Add one to all labels so that sure background is not 0, but 1
    markers = markers + 1

    # Mark the unknown region with 0
    markers[unknown == 255] = 0

    # Apply watershed
    markers = cv2.watershed(image, markers)
    image[markers == -1] = [255, 0, 0]

    return image


In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os

def detect_trees_using_watershed(image_folder, output_folder):
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop through each image in the folder
    for image_file in os.listdir(image_folder):
        image_path = os.path.join(image_folder, image_file)
        image = cv2.imread(image_path)

        if image is None:
            print(f"Unable to load image: {image_path}")
            continue

        # Convert the image to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Apply a threshold to create a binary image
        _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

        # Noise removal using morphological operations
        kernel = np.ones((3, 3), np.uint8)
        opening = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel, iterations=2)

        # Sure background area (dilation)
        sure_bg = cv2.dilate(opening, kernel, iterations=3)

        # Finding sure foreground area using distance transform
        dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
        _, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)

        # Finding unknown region
        sure_fg = np.uint8(sure_fg)
        unknown = cv2.subtract(sure_bg, sure_fg)

        # Marker labelling
        _, markers = cv2.connectedComponents(sure_fg)

        # Add 1 to all markers to distinguish background from the unknown region
        markers = markers + 1

        # Mark the unknown region as zero
        markers[unknown == 255] = 0

        # Apply the watershed algorithm
        markers = cv2.watershed(image, markers)

        # Boundaries detected by watershed will be marked as -1
        image[markers == -1] = [0, 0, 255]

        # Save the output
        output_image_path = os.path.join(output_folder, image_file)
        cv2.imwrite(output_image_path, image)

        # Optionally display the result
        plt.figure(figsize=(8, 8))
        plt.title('Watershed Segmentation Result')
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()

# Example usage
image_folder = "wildfire-week3-v3/train/original"
output_folder = "wildfire-week3-v3/train/alive"
detect_trees_using_watershed(image_folder, output_folder)


In [ ]:
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt

def detect_alive_trees_in_images(image_folder, output_folder):
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop through each image in the folder
    for image_file in os.listdir(image_folder):
        # Load the image
        image_path = os.path.join(image_folder, image_file)
        image = cv2.imread(image_path)
        
        if image is None:
            print(f"Unable to load image: {image_path}")
            continue

        # Convert the image to HSV for color segmentation
        hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

        # Define HSV color range for green (live trees)
        lower_green = np.array([35, 40, 40])
        upper_green = np.array([85, 255, 255])

        # Create a mask for green areas (live trees)
        mask = cv2.inRange(hsv_image, lower_green, upper_green)

        # Apply contour detection on the mask
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Draw contours on the original image to show detected tree crowns
        result_image = image.copy()
        cv2.drawContours(result_image, contours, -1, (0, 255, 0), 2)  # Green color for contours

        # Save the mask and the result image
        mask_output_path = os.path.join(output_folder, image_file.replace('.jpg', '_mask.jpg'))
        result_output_path = os.path.join(output_folder, image_file.replace('.jpg', '_result.jpg'))

        cv2.imwrite(mask_output_path, mask)
        cv2.imwrite(result_output_path, result_image)

        # Optionally, display the result
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        plt.title('Original Image with Detected Tree Crowns')
        plt.imshow(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB))

        plt.subplot(1, 2, 2)
        plt.title('Mask of Alive Trees')
        plt.imshow(mask, cmap='gray')

        plt.show()

# Example usage

output_folder_contour = 'wildfire-week3-v3/train/alive_contour'
detect_alive_trees_in_images(image_folder, output_folder)


In [ ]:
import cv2
import numpy as np
from skimage.feature import graycomatrix, graycoprops
import os
import matplotlib.pyplot as plt

def calculate_texture_features(gray_image):
    # Compute GLCM for various angles and distances
    distances = [1]
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    glcm = graycomatrix(gray_image, distances, angles, symmetric=True, normed=True)

    # Calculate texture features from GLCM
    contrast = graycoprops(glcm, 'contrast')[0]
    dissimilarity = graycoprops(glcm, 'dissimilarity')[0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0]
    energy = graycoprops(glcm, 'energy')[0]
    correlation = graycoprops(glcm, 'correlation')[0]
    entropy = -np.sum(glcm * np.log(glcm + np.finfo(float).eps))

    return contrast, dissimilarity, homogeneity, energy, correlation, entropy

def detect_trees_with_texture(image_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for image_filename in os.listdir(image_folder):
        image_path = os.path.join(image_folder, image_filename)
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error loading image: {image_filename}")
            continue

        # Convert to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian Blur
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)

        # Calculate texture features
        contrast, dissimilarity, homogeneity, energy, correlation, entropy = calculate_texture_features(blurred)

        # Example thresholding based on one of the texture features (adjust as needed)
        if contrast.all() > 0.6:  # This threshold value is arbitrary and should be tuned
            mask = cv2.threshold(blurred, 128, 255, cv2.THRESH_BINARY)[1]
        else:
            mask = cv2.threshold(blurred, 200, 255, cv2.THRESH_BINARY)[1]

        # Morphological operations to clean up the mask
        kernel = np.ones((3, 3), np.uint8)
        cleaned_mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

        # Apply the mask to the original image
        masked_image = cv2.bitwise_and(image, image, mask=cleaned_mask)

        # Save the output mask
        output_mask_path = os.path.join(output_folder, f"tree_mask_{image_filename}")
        cv2.imwrite(output_mask_path, cleaned_mask)

        # Visualize the original image and the masked image
        plt.figure(figsize=(12, 6))

        plt.subplot(1, 2, 1)
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        plt.title(f"Original Image: {image_filename}")

        plt.subplot(1, 2, 2)
        plt.imshow(cv2.cvtColor(masked_image, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        plt.title("Masked Image")

        plt.show()

# Example usage

output_folder = 'wildfire-week3-v3/train/all_trees'  # Change to your output folder path
detect_trees_with_texture(image_folder, output_folder)


In [ ]:
import cv2
import numpy as np
from skimage.feature import graycomatrix, graycoprops
import os
import matplotlib.pyplot as plt

def texture_analysis(image_gray):
    """Perform texture analysis using GLCM and return a texture mask."""
    glcm = graycomatrix(image_gray, distances=[5], angles=[0], levels=256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')
    texture_mask = np.zeros_like(image_gray, dtype=np.uint8)
    
    # Threshold on texture (adjust based on the specific image and texture properties)
    texture_mask[contrast[0, 0] > 0.1] = 255
    
    return texture_mask

def color_segmentation(image):
    """Perform color-based segmentation and return a color mask."""
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_green = np.array([35, 40, 40])
    upper_green = np.array([85, 255, 255])
    color_mask = cv2.inRange(hsv, lower_green, upper_green)
    return color_mask

def apply_transparent_mask(image, mask, alpha=0.4):
    """Apply a transparent mask to the original image."""
    overlay = image.copy()
    
    # Colorize the mask (green in this case, but you can adjust the color)
    mask_colored = np.zeros_like(image)
    mask_colored[:, :, 1] = mask  # Apply mask to the green channel
    
    # Apply the transparent overlay (alpha blending)
    cv2.addWeighted(mask_colored, alpha, overlay, 1 - alpha, 0, overlay)
    
    return overlay

def detect_trees(image):
    """Detect trees in the image using color and texture masks."""
    color_mask = color_segmentation(image)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    texture_mask = texture_analysis(gray_image)
    combined_mask = cv2.bitwise_and(color_mask, texture_mask)
    
    # Morphological operations to clean the mask
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    cleaned_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, kernel)
    
    return cleaned_mask

def process_image_folder(folder_path, output_folder):
    """Process all images in a folder, apply transparent masks, and save the results."""
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for file_name in os.listdir(folder_path):
        if file_name.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(folder_path, file_name)
            image = cv2.imread(image_path)
            
            if image is not None:
                # Detect trees and get the mask
                tree_mask = detect_trees(image)
                
                # Apply transparency mask to the original image
                result_image = apply_transparent_mask(image, tree_mask)
                
                # Save the result
                result_path = os.path.join(output_folder, f"masked_{file_name}")
                cv2.imwrite(result_path, result_image)
                print(f"Processed and saved: {result_path}")
                
                # Optionally display the result
                display_image(image, result_image)
            else:
                print(f"Error reading image: {image_path}")





folder_path = 'wildfire-week4/test/original'
output_folder = 'wildfire-week4/test/all_trees'

process_image_folder(folder_path, output_folder)


In [ ]:
def display_image(original, masked):
    """Display the original and masked image using Matplotlib."""
    plt.figure(figsize=(10, 5))
    
    # Original image
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
    plt.title('Original Image')
    plt.axis('off')
    
    # Masked image with transparency
    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(masked, cv2.COLOR_BGR2RGB))
    plt.title('Masked Image with Transparency')
    plt.axis('off')
    
    plt.show()

## Train dataset with data augmentation

In [ ]:
# Directories for your datasets
train_annotation_file_da = 'wildfire-week4-v2/train/_annotations.coco.json'
train_images_dir_da = 'wildfire-week4-v2/train/original'
train_masks_dir_da = 'wildfire-week4-v2/train/mask'

valid_annotation_file_da = 'wildfire-week4-v2/valid/_annotations.coco.json'
valid_images_dir_da = 'wildfire-week4-v2/valid/original'
valid_masks_dir_da = 'wildfire-week4-v2/valid/mask'

test_annotation_file_da = 'wildfire-week4/test/_annotations.coco.json'
test_images_dir_da = 'wildfire-week4-v2/test/original'
test_masks_dir_da = 'wildfire-week4-v2/test/mask'

In [ ]:
generate_mask(train_annotation_file_da, train_images_dir_da, train_masks_dir_da)

In [ ]:
generate_mask(valid_annotation_file_da, valid_images_dir_da, valid_masks_dir_da)

In [ ]:
generate_mask(test_annotation_file_da, test_images_dir_da, test_masks_dir_da)

In [ ]:
unet_model_da = Unet(
    encoder_name="resnet50",  # Choose your backbone (e.g., "resnet34", "efficientnet-b3", etc.)
    encoder_weights="imagenet",  # Use pre-trained weights
    in_channels=3,  # Input channels (e.g., 3 for RGB images)
    classes=5  # Number of output classes
)
input_tensor = torch.randn(1, 3, 640, 640)  # Batch size of 1, 3 channels (RGB), 640x640 image
output = unet_model_da(input_tensor)
print(output.shape)

In [ ]:
# Create datasets
train_dataset_da = SegmentationDataset(train_images_dir_da, train_masks_dir_da, transform)
valid_dataset_da = SegmentationDataset(valid_images_dir_da, valid_masks_dir_da, transform)
test_dataset_da = SegmentationDataset(test_images_dir_da, test_masks_dir_da, transform)

# Create data loaders
train_loader_da = DataLoader(train_dataset_da, batch_size=4, shuffle=True)
valid_loader_da = DataLoader(valid_dataset_da, batch_size=4, shuffle=False)
test_loader_da = DataLoader(test_dataset_da, batch_size=4, shuffle=False)

In [ ]:
coco_json_path_da = "wildfire-week4-v2/train/_annotations.coco.json"
image_directory_da = "wildfire-week4-v2/train/original"
class_counts = count_classes_in_coco(coco_json_path_da, image_directory_da)
print(class_counts)

In [ ]:
class_weights_da = calculate_class_weights(class_counts)
print("Class Weights:", class_weights_da)

# You can now use these weights in nn.CrossEntropyLoss
criterion_da = torch.nn.CrossEntropyLoss(weight=class_weights_da)


In [ ]:
optimizer = optim.Adam(unet_model_da.parameters(), lr=1e-4)
train_model(unet_model_da, train_loader_da, valid_loader_da, criterion_da, optimizer, num_epochs=10, patience=5)

In [ ]:
average_loss, pixel_accuracy, results_table = evaluate_model_classname(unet_model_da, test_loader_da,torch.nn.CrossEntropyLoss() , class_names)
print(results_table)

In [ ]:
output_list = visualize_predictions_with_ytrue(test_loader_da, unet_model, num_samples=3)

In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

def calculate_exg(image):
    # Convert the image from BGR to RGB
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Split the image into R, G, and B channels
    R = rgb_image[:, :, 0].astype(float)
    G = rgb_image[:, :, 1].astype(float)
    B = rgb_image[:, :, 2].astype(float)
    
    # Calculate Excess Green Index (ExG)
    ExG = (2 * G - R - B) / (2 * G + R + B + 1e-10)  # Adding a small constant to avoid division by zero

    return ExG

def apply_mask(original_image, exg_image, alpha=0.5):
    # Normalize ExG for masking
    exg_image_normalized = cv2.normalize(exg_image, None, 0, 255, cv2.NORM_MINMAX)
    exg_image_normalized = exg_image_normalized.astype(np.uint8)

    # Calculate quantile-based threshold for trees
    threshold_value_tree = np.quantile(exg_image_normalized, 0.6)  # 75th percentile for trees

    # Create a binary mask for trees
    tree_mask = np.zeros_like(exg_image_normalized)
    tree_mask[exg_image_normalized > threshold_value_tree] = 255  # Trees

    # Create RGBA image for visualization
    rgba_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGBA)

    # Create a color image for the tree mask
    tree_color = np.zeros_like(rgba_image)
    tree_color[tree_mask == 255] = [0, 255, 0, 255]  # Green for trees

    # Convert the tree mask to have an alpha channel
    tree_color = cv2.cvtColor(tree_color, cv2.COLOR_BGR2BGRA)

    # Blend the original image with the tree mask using transparency
    blended_image = cv2.addWeighted(rgba_image, 1 - alpha, tree_color, alpha, 0)

    return tree_mask, blended_image


def visualize_images(original_image, mask, masked_image):
    # Create a figure to visualize the original and masked images
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    plt.imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
    plt.title('Original Image')
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.imshow(mask, cmap='gray')
    plt.title('ExG Mask')
    plt.axis('off')
    
    plt.subplot(1, 3, 3)
    plt.imshow(cv2.cvtColor(masked_image, cv2.COLOR_BGR2RGB))
    plt.title('Masked Image')
    plt.axis('off')

    plt.show()

def process_images_in_folder(input_folder, output_folder):
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Iterate through all files in the input folder
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):  # Check for image files
            # Construct the full file path
            file_path = os.path.join(input_folder, filename)

            # Read the image
            original_image = cv2.imread(file_path)
            if original_image is None:
                print(f"Could not read image: {file_path}")
                continue

            # Calculate ExG
            exg_image = calculate_exg(original_image)

            # Apply mask to the original image
            mask, masked_image = apply_mask(original_image, exg_image)

            # Save the ExG mask
            output_file_path_mask = os.path.join(output_folder, f"exg_mask_{filename}")
            cv2.imwrite(output_file_path_mask, mask)
            print(f"Saved ExG mask: {output_file_path_mask}")

            # Visualize the images
            visualize_images(original_image, mask, masked_image)



# Set the input and output folder paths
input_folder = 'wildfire-week4/train/original'  # Change this to your input folder
output_folder = 'wildfire-week4/train/exg'  # Change this to your desired output folder

# Process the images in the specified folder
process_images_in_folder(input_folder, output_folder)
